In [26]:
import pandas as pd
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import tensorflow as tf

2022-11-13 22:48:10.774195: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-13 22:48:10.913667: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-11-13 22:48:10.917725: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-13 22:48:10.917738: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if yo

In [5]:
dir = "/home/whfhrs3260/csv_data/"

In [6]:
score = pd.read_csv(dir+"/price_data_score.csv")
score.columns = ["Date", "Score", "class_3", "class_5"]
score = score[["Date","class_3"]]

In [8]:
price_data = yf.download(["^DJI"],start = '2017-01-01', end = "2022-05-01")
price_data = price_data.reset_index()

[*********************100%***********************]  1 of 1 completed


In [10]:
first_data = yf.download(["^DJI"],start = '2016-12-30', end = '2017-01-01')

[*********************100%***********************]  1 of 1 completed


In [11]:
first_data

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2016-12-30,19833.169922,19852.550781,19718.669922,19762.599609,19762.599609,271910000


In [12]:
def time_conversion(time):
    time = pd.Timestamp(time)
    return time

score.Date = score.Date.apply(lambda x: time_conversion(x))

In [13]:
price_data = pd.merge(price_data, score, how ="left",left_on='Date', right_on = "Date")[["Close","Open","High","Low","Adj Close","class_3"]]
price_data.class_3[price_data.class_3.isnull()]=0
price_data["before_close"] = 0
price_data = price_data.reset_index(drop=True)
for i in range(len(price_data)-1):
    price_data.before_close[i+1] = price_data.Close[i]
    
price_data.before_close[0] = first_data["Close"][0]

for i in range(len(price_data)):
    price_data.High[len(price_data)-i] =price_data.High[len(price_data)-i-1]
    price_data.Open[len(price_data)-i] =price_data.Open[len(price_data)-i-1]
    price_data.Low[len(price_data)-i] =price_data.Low[len(price_data)-i-1]
    price_data["Adj Close"][len(price_data)-i] =price_data["Adj Close"][len(price_data)-i-1]

price_data.High[0] = first_data["High"][0]
price_data.Open[0] = first_data["Open"][0]
price_data.Low[0] = first_data["Low"][0]
price_data["Adj Close"][0] = first_data["Adj Close"][0]

price_data.columns = ["Close","before_Open","before_High","before_Low","before_Adj Close","class_3","before_close"]

/tmp/ipykernel_30923/2391332866.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  price_data.before_close[i+1] = price_data.Close[i]


In [17]:
x = price_data[["class_3","before_close"]]
y = price_data[["Close"]]
scaler_x = MinMaxScaler()
scaler_y = MinMaxScaler()

x = pd.DataFrame(scaler_x.fit_transform(x))
y = pd.DataFrame(scaler_y.fit_transform(y))

x.columns = ["class_3","before_close"]
y.columns = ["Close"]

train_index = int(len(x)*0.7)

train_x = x.iloc[0:train_index]
test_x = x.iloc[train_index:len(x)]

train_y = y.iloc[0:train_index]
test_y = y.iloc[train_index:len(y)]

In [18]:
train_x = train_x.to_numpy().reshape(train_x.shape[0],1,train_x.shape[1])
train_y = train_y.to_numpy().reshape(train_y.shape[0],train_y.shape[1])
test_x = test_x.to_numpy().reshape(test_x.shape[0],1,test_x.shape[1])
test_y = test_y.to_numpy().reshape(test_y.shape[0],test_y.shape[1])

In [22]:
window_size = 30

In [23]:
x = np.zeros(shape=(train_x.shape[0]-window_size+1,window_size,x.shape[1]))
for i in range(train_x.shape[0]-window_size+1):
    x[i]=np.vstack((train_x[i:i+window_size]))

y = train_y[window_size-1:train_x.shape[0]]

x_t = np.zeros(shape=(test_x.shape[0]-window_size+1,window_size,x.shape[2]))
for i in range(test_x.shape[0]-window_size+1):
    x_t[i]=np.vstack((test_x[i:i+window_size]))
    
y_t = test_y[window_size-1:test_x.shape[0]]

In [49]:
generator = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(1024,1)),
    tf.keras.layers.GRU(1024,return_sequences=True),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.GRU(512,return_sequences=True),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.GRU(256),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(128),
    tf.keras.layers.Dense(64),
    tf.keras.layers.Dense(1)])

In [52]:
discriminator = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(935,2)),
    tf.keras.layers.Conv1D(32,3),
    tf.keras.layers.LeakyReLU(0.01),
    tf.keras.layers.Conv1D(64,5),
    tf.keras.layers.LeakyReLU(0.01),
    tf.keras.layers.Conv1D(128,5),
    tf.keras.layers.LeakyReLU(0.01),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(220),
    tf.keras.layers.LeakyReLU(),
    tf.keras.layers.Dense(220, activation="relu"),
    tf.keras.layers.Dense(1, activation = "sigmoid")])


In [ ]:
def train(input):
    with tf.GradientTape() as t1, tf.GradientTape() as t2:
        g = generator(tf.random.normal([1, 1024]))
        g_d = discriminator(g)
        d = discriminator(input)
        
        loss_d = -tf.reduce_mean(tf.math.log(d) + tf.math.log(1-g_d))
        loss_g = -tf.reduce_mean(tf.math.log(g_d))
        
    d_upgrade = t1.gradient(loss_d, discriminator.trainable_variables)
    g_upgrade = t2.gradient(loss_g, generator.trainable_variables)